In [ ]:
import subprocess
import sys

def install_package(package):
    """Install a package using pip if it's not already installed."""
    try:
        __import__(package)
        print(f"✅ Requirement already satisfied: {package}")
    except ImportError:
        print(f"📦 Installing {package}...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"✅ Successfully installed {package}")
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed to install {package}. Error: {e}")
            # Exit if a critical package fails to install
            sys.exit(1)

# List of all required Python packages for the code provided:
required_packages = [
    'flyr',  # The 'flyr' library
    'pandas',
    'watchdog',
    'matplotlib',
    'sqlalchemy',
    'pyodbc'          # Driver for MS SQL connectivity
]

for pkg in required_packages:
    install_package(pkg)

print("\nPython dependencies satisfied. ensure 'exiftool-12.35.exe' is in the script directory.")
print("Download ODBC Driver 17 for SQL Server to be able to upload: https://go.microsoft.com/fwlink/?linkid=2266337")

✅ Requirement already satisfied: flyr
✅ Requirement already satisfied: pandas
✅ Requirement already satisfied: watchdog
✅ Requirement already satisfied: matplotlib
✅ Requirement already satisfied: sqlalchemy
✅ Requirement already satisfied: pyodbc

Python dependencies satisfied. ensure 'exiftool-12.35.exe' is in the script directory.
Download ODBC Driver 17 for SQL Server to be able to upload: https://go.microsoft.com/fwlink/?linkid=2266337


In [ ]:
################################### THERMAL DATA EXTRACTOR - MSSQL UPLOAD ####################
import os
import json
import subprocess
import flyr
import matplotlib.pyplot as plt
import base64
import io
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# --- CONFIGURATION ---
INPUT_FOLDER = "flir e5 photodump"
EXIFTOOL_PATH = "exiftool-12.35.exe" 

# --- DATABASE CREDENTIALS ---
DB_SERVER = "PSQLAPPEG297-01"
DB_NAME = "Flir"
DB_USER = "Flir"
DB_PASS = "Prom@2025"
DB_TABLE = "ThermalReadings"

# --- HELPER FUNCTIONS ---
def get_db_engine():
    encoded_pass = quote_plus(DB_PASS)
    db_url = f"mssql+pyodbc://{DB_USER}:{encoded_pass}@{DB_SERVER}/{DB_NAME}?driver=ODBC+Driver+17+for+SQL+Server"
    return create_engine(db_url)

def get_existing_signatures(engine, start_date_str):
    """
    SMART FILTER: Queries the DB starting exactly from the oldest image in your batch.
    start_date_str format: 'YYYY-MM-DD HH:MM:SS'
    """
    try:
        # We query for anything NEWER than or EQUAL to the oldest image in our folder
        query = f"""
            SELECT Asset_Name, Timestamp 
            FROM {DB_TABLE} 
            WHERE Timestamp >= '{start_date_str}'
        """
        
        df = pd.read_sql(query, engine)
        
        if not df.empty:
            df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='mixed')
            signatures = set(zip(
                df['Asset_Name'], 
                df['Timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
            ))
            return signatures
        return set()
    except Exception as e:
        print(f"⚠️ Warning: Could not check duplicates (Database might be empty): {e}")
        return set()

def get_metadata(folder):
    cmd = [
        EXIFTOOL_PATH, '-j', '-n', '-r', 
        '-DateTimeOriginal', 
        '-CameraSerialNumber', 
        '-ImageDescription', 
        '-Emissivity', '-ObjectDistance', '-ext', 'jpg', folder
    ]
    try:
        flags = subprocess.CREATE_NO_WINDOW if os.name == 'nt' else 0
        result = subprocess.run(cmd, capture_output=True, text=True, creationflags=flags)
        return json.loads(result.stdout)
    except Exception as e:
        print(f"Metadata scan failed: {e}")
        return []

def process_image(filepath, metadata_entry):
    filename = os.path.basename(filepath)
    asset_name = metadata_entry.get("ImageDescription")
    asset_str = str(asset_name).strip() if asset_name else ""
    
    if not asset_str: return None

    serial_int = int(metadata_entry["CameraSerialNumber"])
    ts_str = str(metadata_entry["DateTimeOriginal"]).replace(":", "-", 2)

    row = {
        "Timestamp": ts_str, 
        "Filename": filename,
        "Camera_Serial": serial_int,
        "Asset_Name": asset_str,     
        "Max_Temp_C": 0.0, "Min_Temp_C": 0.0, "Center_Temp_C": 0.0,        
        "Avg_Temp_C": 0.0, "Delta_Temp_C": 0.0, "Emissivity": 0.95, "Distance": 1.0,             
        "Image_Base64": ""
    }

    if metadata_entry.get("Emissivity"): row["Emissivity"] = float(metadata_entry["Emissivity"]) 
    if metadata_entry.get("ObjectDistance"): row["Distance"] = round(float(metadata_entry["ObjectDistance"]), 1)

    try:
        thermogram = flyr.unpack(filepath)
        celsius = thermogram.celsius
        
        row["Max_Temp_C"] = round(celsius.max(), 1)
        row["Min_Temp_C"] = round(celsius.min(), 1)
        row["Avg_Temp_C"] = round(celsius.mean(), 1)
        row["Delta_Temp_C"] = round(celsius.max() - celsius.min(), 1)
        
        h, w = celsius.shape
        cy, cx = h // 2, w // 2
        row["Center_Temp_C"] = round(celsius[cy-1:cy+2, cx-1:cx+2].mean(), 1)

        buffer = io.BytesIO()
        plt.imsave(buffer, celsius, cmap='inferno', format='jpeg')
        buffer.seek(0)
        raw_b64 = base64.b64encode(buffer.getvalue()).decode('utf-8')
        row["Image_Base64"] = f"data:image/jpeg;base64,{raw_b64}"
        buffer.close()
    except Exception as e:
        print(f"Error processing {filename}: {e}")
        return None 
    return row

# --- MAIN ---
def main():
    if not os.path.exists(INPUT_FOLDER):
        print(f"Folder not found: {INPUT_FOLDER}")
        return

    # 1. First, scan Metadata locally to find the time range
    print("Scanning folder metadata...")
    meta_list = get_metadata(INPUT_FOLDER)
    
    meta_dict = {}
    timestamps = []
    
    for m in meta_list:
        if 'SourceFile' in m:
            fname = os.path.basename(m['SourceFile'])
            meta_dict[fname] = m
            # Collect all valid timestamps
            if 'DateTimeOriginal' in m:
                # Convert "YYYY:MM:DD HH:MM:SS" to "YYYY-MM-DD HH:MM:SS"
                ts = str(m['DateTimeOriginal']).replace(":", "-", 2)
                timestamps.append(ts)
    
    if not timestamps:
        print("No images with timestamps found in folder.")
        return

    # 2. Determine the Dynamic Window
    # Find the oldest photo in the batch
    oldest_photo_time = min(timestamps)
    # We strip the time part to query the whole day, just to be safe
    query_start_date = oldest_photo_time[:10] + " 00:00:00"
    
    print(f"Detected oldest image from: {query_start_date}")
    print(f"Connecting to DB to check records since {query_start_date}...")

    # 3. Connect & Get Duplicates
    try:
        engine = get_db_engine()
        existing_signatures = get_existing_signatures(engine, query_start_date)
        print(f"Database holds {len(existing_signatures)} potential duplicates in this time range.")
    except Exception as e:
        print(f"❌ Connection failed: {e}")
        return
    
    # 4. Filter New Files
    files = [f for f in os.listdir(INPUT_FOLDER) if f.lower().endswith(".jpg")]
    files_to_process = []
    skipped_duplicates = 0
    
    for f in files:
        m_data = meta_dict.get(f, {})
        raw_asset = m_data.get("ImageDescription")
        raw_time = m_data.get("DateTimeOriginal")
        
        if raw_asset and raw_time:
            asset_key = str(raw_asset).strip()
            time_full = str(raw_time).replace(":", "-", 2)
            time_key_simple = time_full[:19] 
            
            if (asset_key, time_key_simple) in existing_signatures:
                skipped_duplicates += 1
                continue 
            else:
                files_to_process.append(f)
        else:
            files_to_process.append(f)

    if not files_to_process:
        print(f"\nNo new data found. (Skipped {skipped_duplicates} duplicates).")
        return

    print(f"Found {len(files_to_process)} NEW images to upload...")

    # 5. Process & Upload
    new_data_rows = []
    for i, f in enumerate(files_to_process):
        full_path = os.path.join(INPUT_FOLDER, f)
        row = process_image(full_path, meta_dict.get(f, {}))
        if row:
            new_data_rows.append(row)
            print(f"[{i+1}/{len(files_to_process)}] PROCESSED: {f}")

    if new_data_rows:
        cols = [
            "Timestamp", "Filename", "Camera_Serial", "Asset_Name", 
            "Max_Temp_C", "Min_Temp_C", "Center_Temp_C", "Avg_Temp_C", "Delta_Temp_C",
            "Emissivity", "Distance", "Image_Base64"
        ]
        new_df = pd.DataFrame(new_data_rows, columns=cols)
        # Ensure timestamp format for SQL
        new_df['Timestamp'] = pd.to_datetime(new_df['Timestamp'], format='mixed')

        print(f"Uploading {len(new_df)} rows to table '{DB_TABLE}'...")
        try:
            new_df.to_sql(DB_TABLE, engine, if_exists='append', index=False)
            print("✅ Upload Successful!")
        except Exception as e:
            print(f"❌ Database Upload Failed: {e}")
    else:
        print("\nNo valid new images were found.")

if __name__ == "__main__":
    main()